In [ ]:
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import tempfile
import os
from datetime import datetime

def post_to_linkedin(email, password, message, image_paths):
    options = Options()
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    try:
        driver.get("https://www.linkedin.com/login")
        time.sleep(3)
        driver.find_element(By.ID, "username").send_keys(email)
        driver.find_element(By.ID, "password").send_keys(password)
        driver.find_element(By.XPATH, "//button[@type='submit']").click()
        time.sleep(5)
        driver.get("https://www.linkedin.com/feed/")
        time.sleep(5)
        driver.find_element(By.CLASS_NAME, "share-box-feed-entry__trigger").click()
        time.sleep(3)
        post_area = driver.find_element(By.CLASS_NAME, "ql-editor")
        post_area.send_keys(message)
        time.sleep(2)
        if image_paths:
            for path in image_paths:
                upload_input = driver.find_element(By.XPATH, '//input[@type="file"]')
                upload_input.send_keys(path)
                time.sleep(2)
        driver.find_element(By.XPATH, "//button/span[text()='Post']").click()
        time.sleep(5)
        driver.quit()
        return "Post published successfully."
    except Exception as e:
        driver.quit()
        return f"Failed: {str(e)}"

st.title("LinkedIn Auto Poster")

email = st.text_input("LinkedIn Email")
password = st.text_input("LinkedIn Password", type="password")
message = st.text_area("Post Message")
images = st.file_uploader("Upload Images", accept_multiple_files=True, type=["jpg", "jpeg", "png"])
scheduled_time = st.time_input("Schedule time", value=datetime.now().time())

if st.button("Schedule & Post"):
    if not email or not password or not message:
        st.warning("Please fill all required fields.")
    else:
        now = datetime.now()
        scheduled_datetime = datetime.combine(now.date(), scheduled_time)
        wait_seconds = (scheduled_datetime - now).total_seconds()
        if wait_seconds > 0:
            st.info(f"Waiting {int(wait_seconds)} seconds until scheduled time...")
            time.sleep(wait_seconds)
        temp_image_paths = []
        for uploaded_file in images:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as tmp:
                tmp.write(uploaded_file.read())
                temp_image_paths.append(tmp.name)
        result = post_to_linkedin(email, password, message, temp_image_paths)
        st.success(result)
        for path in temp_image_paths:
            os.remove(path)